In [1]:
% matplotlib inline

import numpy as np
import pandas as pd
from math import *
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf, ccf, acovf
from statsmodels.tsa.tsatools import detrend
from scipy.spatial.distance import pdist, squareform

import gsw as sw

from mpl_toolkits.basemap import Basemap, cm
from netCDF4 import Dataset as NetCDFFile

/Users/sclayton/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def SVh( P, h, bw ):
    '''
    Experimental semivariogram for a single lag
    '''
    pd = squareform( pdist( P[:,:2] ) )
    N = pd.shape[0]
    Z = list()
    for i in range(N):
        for j in range(i+1,N):
            if( pd[i,j] >= h-bw )and( pd[i,j] <= h+bw ):
                Z.append( ( P[i,2] - P[j,2] )**2.0 )
    return np.sum( Z ) / ( 2.0 * len( Z ) )
 
def SV( P, hs, bw ):
    '''
    Experimental variogram for a collection of lags
    '''
    sv = list()
    for h in hs:
        sv.append( SVh( P, h, bw ) )
    sv = [ [ hs[i], sv[i] ] for i in range( len( hs ) ) if sv[i] > 0 ]
    return np.array( sv ).T
 
def C( P, h, bw ):
    '''
    Calculate the sill
    '''
    c0 = np.var( P[:,2] )
    if h == 0:
        return c0
    return c0 - SVh( P, h, bw )

def opt( fct, x, y, C0, parameterRange=None, meshSize=1000 ):
    if parameterRange == None:
        parameterRange = [ x[1], x[-1] ]
    mse = np.zeros( meshSize )
    a = np.linspace( parameterRange[0], parameterRange[1], meshSize )
    for i in range( meshSize ):
        mse[i] = np.mean( ( y - fct( x, a[i], C0 ) )**2.0 )
    return a[ mse.argmin() ]

def spherical(h, a, C0):
    '''
    Spherical model of the semivariogram
    '''
    # if h is a single digit
    if type(h) == np.float64:
        # calculate the spherical function
        if h <= a:
            return C0*(1.5*(h/a) - (1/3)*((h**3.0)/(a**3.0)))
        else:
            return C0
    # if h is an iterable
    else:
        # calculate the spherical function for all elements
        a = np.ones( h.size ) * a
        C0 = np.ones( h.size ) * C0
        return map(spherical, h, a, C0)
    
def cvmodel(P, model, hs, bw):
    '''
    Input:  (P)      ndarray, data
            (model)  modeling function
                      - spherical
                      - exponential
                      - gaussian
            (hs)     distances
            (bw)     bandwidth
    Output: (covfct) function modeling the covariance
    '''
    # calculate the semivariogram
    sv = SV( P, hs, bw )
    # calculate the sill
    C0 = C(P, hs[0], bw)
    # calculate the optimal parameters
    param = opt( model, sv[0], sv[1], C0 )
    # return a covariance function
    covfct = lambda h, a=param: C0 - model( h, a, C0 )
    return covfct

In [3]:
# import data

maxlon = 170.01
minlon = 139

Tok1 = pd.read_csv(r'/Users/sclayton/Google Drive/o2ar_data/Tokyo1_vSept16.csv', sep = ',')
Tok1['density'] = sw.rho(Tok1['S'].values,Tok1['T'].values,0)
Tok1 = Tok1[(Tok1['Lon']>minlon) & (Tok1['Lon']<maxlon)]

Tok1['dlat'] = np.insert(np.diff(Tok1['Lat']), 0, 0)
Tok1 = Tok1[np.abs(Tok1['dlat'])<0.08]

Tok3 = pd.read_csv(r'/Users/sclayton/Google Drive/o2ar_data/Tokyo3_vSept16.csv', sep = ',')
Tok3['density'] = sw.rho(Tok3['S'].values,Tok3['T'].values,0)
Tok3 = Tok3[(Tok3['Lon']>minlon) & (Tok3['Lon']<maxlon)]

Tian = pd.read_csv(r'/Users/sclayton/Google Drive/o2ar_data/Tianjin2_vSept16.csv', sep = ',')
Tian['density'] = sw.rho(Tian['S'].values,Tian['T'].values,0)
Tian = Tian[(Tian['Lon']>minlon) & (Tian['Lon']<maxlon)]

print Tian.columns

Index([u'Timestamp', u'Lat', u'Lon', u'S', u'T', u'Fluo', u'TrueO2Ar',
       u'O2Arsat', u'O2Arbiosat', u'MLD (WOA13)', u'Wkn', u'Air-sea flux',
       u'density'],
      dtype='object')


In [5]:
# calculate distance between points
Tok1['distance'] = np.insert(np.cumsum(sw.distance(Tok1['Lon'], Tok1['Lat'], 0)[0]),0,0)/1000

print Tok1.distance

838        0.000000
839        2.069866
840        4.157046
841        6.259494
842        8.320467
843       10.411399
845       41.342079
846       43.330987
847       45.372740
848       47.361369
849       49.305074
850       51.307134
851       53.272248
852       55.250219
853       57.199284
854       59.110004
855       61.046766
856       62.942924
857       64.844032
858       66.740524
859       68.619094
860       70.542546
861       72.411930
862       74.287622
863       76.166047
864       78.071882
865       80.045039
866       81.976192
867       83.907883
868       85.805186
           ...     
1930    2865.279468
1931    2867.252774
1932    2869.165978
1933    2871.127113
1934    2873.086109
1935    2874.986654
1936    2876.916568
1937    2878.867059
1938    2880.823455
1939    2882.737268
1940    2884.722904
1941    2886.688679
1942    2888.637604
1943    2890.642999
1944    2892.590566
1945    2894.553373
1946    2896.554308
1947    2898.533014
1948    2900.546628


In [ ]:
# automate process of getting continuous segments
# make this a function

segments = []

start = 0
for i in range(len(crsub)-1):
    
    if cdistance[i] > 20000:

        s = crsub[start:i+1]
        print "found", i, cdistance[i], start
        #raw_input()
        segments.append(s)
        start = i+1
                                             
print [s.shape for s in segments]